In [1]:
import joblib
import numpy as np
import pandas as pd
from sklearn import metrics, model_selection, preprocessing
import xgboost as xgb

from pan20 import util

In [2]:
df = pd.read_csv('data/fake/feats.csv')

In [3]:
def get_X_y(cols=None, no_cols=None):
    X = df.groupby(['author', 'label']).mean().reset_index()
    y = X.label.values
    a = list(X.author.values)
    X.drop(columns=['author', 'n'], inplace=True)
    if cols:
        X = X.loc[:, [c in cols for c in X.columns]]
    elif no_cols:
        X = X.loc[:, [c not in no_cols for c in X.columns]]
    else:
        X = X.loc[:, [c != 'label' for c in X.columns]]
    feat_dict = util.IxDict(X.columns)
    # normalize features to be between 0 and 1
    x = X.values
    min_max_scaler = preprocessing.MinMaxScaler()
    X = min_max_scaler.fit_transform(x)
    return X, y, feat_dict, a

In [4]:
feats = [
    'adverbs', 
    'impersonal_pronouns',
    'personal_pronouns', 
    'function_words',
    'avg_bf', 
    'max_np_height', 
    'max_vp_height',
    'senti', 
    'senti_neg', 
    'anger', 
    'distrust'
]

In [5]:
X, y, feat_dict, a = get_X_y(feats)

In [6]:
svc = joblib.load('pan20/fake/svc.model')
rf = joblib.load('pan20/fake/rf.model')
nb = joblib.load('pan20/fake/nb.model')

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SVC from version 0.22.2.post1 when using version 0.22.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.2.post1 when using version 0.22.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.22.2.post1 when using version 0.22.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SVC from version 0.22.2.post1 when using version 0.22.2. This might lead 

In [7]:
p_svc = list(np.exp(svc.predict_log_proba(X))[:, 1])
p_rf = list(np.exp(rf.predict_log_proba(X))[:, 1])
p_nb = list(np.exp(nb.predict_log_proba(X))[:, 1])

In [146]:
#p_bert = pd.read_csv('tmp/bert_encoded.csv')
#p_bert = pd.read_csv('tmp/bert_large_svm_070.csv')
p_bert = pd.read_csv('tmp/bert_large_svm_070.csv')
p_roberta = pd.read_csv('tmp/roberta_svm_071.csv')
#p_electra = pd.read_csv('tmp/electra_base_svm_066.csv')

In [147]:
#p_bert.drop(columns=['label'], inplace=True)
p_roberta.drop(columns=['label'], inplace=True)

In [148]:
xgb_in = pd.DataFrame({
    'author': a,
    'label': y,
    'svc': p_svc,
    'rf': p_rf,
    'nb': p_nb,
})

In [149]:
xgb_in.set_index('author')
p_bert.set_index('author')
xgb_in = xgb_in.merge(p_bert, on='author')
xgb_in = xgb_in.merge(p_roberta, on='author')
#xgb_in = xgb_in.merge(p_electra, on='author')

In [142]:
xgb_in.head()

,author,label_x,svc,rf,nb,label_y,probability_x,probability_y
0,06ct0t68y1acizh9eow3g5rhancrppr8,1,0.714161,0.867710,0.706297,1,0.941727,0.686610
1,071nxc49ihpd0jlfmvn2lghtayy3b5n9,0,0.595813,0.445694,0.591051,0,0.842279,0.736981
2,09py5qescynpnnckmzueqzr2y49moh1o,0,0.511316,0.361854,0.509568,0,0.429073,0.244102
3,0dwovd7nj6yg9m795ng2c629me0ccmrh,0,0.475750,0.260155,0.475279,0,0.308907,0.356693
4,0ibi364m7i7l01xi4xqafyathrmrrnll,1,0.680197,0.752722,0.673100,1,0.612581,0.673037


In [150]:
xgb_in.drop(columns=['label_y'], inplace=True)
#xgb_in.rename(columns={'label_x': 'label', 'probability': 'bert'}, inplace=True)
xgb_in.rename(columns={'label_x': 'label', 'probability_x': 'bert', 'probability_y': 'roberta'}, inplace=True)

In [62]:
xgb_in.to_csv('tmp/xgb_in.csv', index=False)

In [151]:
def to_txt(feats, file_path, test=False):
    with open(file_path, 'w+') as f:
        for _, x in feats.iterrows():
            if not test:
                row = f'{x.label} 0:{x.svc} 1:{x.rf} 2:{x.nb} 3:{x.bert} 4:{x.roberta}\n'
            else:
                row = f'0:{x.svc} 1:{x.rf} 2:{x.nb} 3:{x.bert} 4:{x.roberta}\n'
            f.write(row)

In [156]:
def to_txt(feats, file_path, test=False):
    with open(file_path, 'w+') as f:
        for _, x in feats.iterrows():
            if not test:
                row = f'{x.label} 0:{x.svc} 1:{x.rf} 2:{x.nb}\n'
            else:
                row = f'0:{x.svc} 1:{x.rf} 2:{x.nb}\n'
            f.write(row)

In [152]:
accs = []
skf = model_selection.StratifiedKFold(n_splits=5)
for tr_ix, ts_ix in skf.split(X, y):
    tr = xgb_in.iloc[tr_ix]
    ts = xgb_in.iloc[ts_ix]
    to_txt(tr, 'tmp/xgb_train.txt')
    to_txt(ts, 'tmp/xgb_test.txt', test=True)
    dtrain = xgb.DMatrix('tmp/xgb_train.txt')
    dtest = xgb.DMatrix('tmp/xgb_test.txt')
    params = {
        'max_depth': 2,
        'eta': 1,
        'objective': 'binary:logistic',
    }
    num_round = 2
    bst = xgb.train(params, dtrain, num_round)
    preds = bst.predict(dtest)
    preds = [p > 0.5 for p in preds]
    accs.append(metrics.accuracy_score(ts.label.values, preds))
print(accs)
print(np.mean(accs))

[15:57:12] 240x5 matrix with 1200 entries loaded from tmp/xgb_train.txt
[15:57:12] 60x5 matrix with 240 entries loaded from tmp/xgb_test.txt
[15:57:12] 240x5 matrix with 1200 entries loaded from tmp/xgb_train.txt
[15:57:12] 60x5 matrix with 240 entries loaded from tmp/xgb_test.txt
[15:57:12] 240x5 matrix with 1200 entries loaded from tmp/xgb_train.txt
[15:57:12] 60x5 matrix with 240 entries loaded from tmp/xgb_test.txt
[15:57:12] 240x5 matrix with 1200 entries loaded from tmp/xgb_train.txt
[15:57:12] 60x5 matrix with 240 entries loaded from tmp/xgb_test.txt
[15:57:12] 240x5 matrix with 1200 entries loaded from tmp/xgb_train.txt
[15:57:12] 60x5 matrix with 240 entries loaded from tmp/xgb_test.txt
[0.95, 0.9166666666666666, 0.9, 0.8666666666666667, 0.8833333333333333]
0.9033333333333333


In [153]:
xgbtr = xgb_in[['label', 'svc', 'rf', 'nb']]

In [157]:
to_txt(xgbtr, 'tmp/xgb_train.txt', test=False)
to_txt(xgbtr, 'tmp/xgb_test.txt', test=True)
dtrain = xgb.DMatrix('tmp/xgb_train.txt')
dtest = xgb.DMatrix('tmp/xgb_test.txt')

[16:11:33] 300x3 matrix with 900 entries loaded from tmp/xgb_train.txt
[16:11:33] 300x3 matrix with 600 entries loaded from tmp/xgb_test.txt


In [158]:
params = {
    'max_depth': 3,
    'eta': 1,
    'objective': 'binary:logistic',
}
num_round = 2
bst = xgb.train(params, dtrain, num_round)

In [160]:


preds = bst.predict(dtest)
preds = [p > 0.5 for p in preds]
metrics.accuracy_score(xgb_in.label.values, preds)



0.9066666666666666

In [162]:
len(xgb_in.label.values)

300

In [164]:
len(preds)

300